In [ ]:
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, MinMaxScaler
warnings.filterwarnings('ignore')
data = pd.read_csv('''C:\\Users\\user\\Documents\\GitHub\\notebooks-on-ml\\Coursera lil proj\\DATA\\a_steam_data_2021_2025.csv''')

In [ ]:
data['genres'] = data['genres'].fillna(data.groupby('categories')['genres'].transform(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan))
data['genres'] = data['genres'].replace(['nan', 'None', ''], np.nan)
data['categories'] = data['categories'].replace(['nan', 'None', ''], np.nan)
data = data.dropna(subset=['genres', 'categories'])
data['developer'] = data['developer'].fillna('Noname')
data['publisher'] = data['publisher'].fillna('Noname')

In [ ]:
data['release_date'] = pd.to_datetime(data['release_date'], format='mixed', errors='coerce')
data['release_date'] = data['release_date'].fillna(pd.to_datetime('Aug 1 ' + data['release_year'].astype(str), format='%b %d %Y', errors='coerce'))       
data['genres'] = data['genres'].astype(str).str.split(';')
data['categories'] = data['categories'].astype(str).str.split(';')

In [ ]:
data['is_sequel'] = data['name'].apply(lambda x: 1 if any(word.upper() in ['2', '3', 'II', 'III', 'IV', 'V'] for word in str(x).split()) else 0).astype(int)
data['developer&publisher'] = (data['developer'] == data['publisher'].values).astype(int)

data['genre_count'] = data['genres'].apply(len)
data['categories_count'] = data['categories'].apply(len)

data['niche'] = data['genres'].str[0] + '_' + data['categories'].str[0]
data['unique_genres'] = data['developer'].map(data.explode('genres').groupby('developer')['genres'].nunique())
data['unique_categories'] = data['developer'].map(data.explode('categories').groupby('developer')['categories'].nunique())

data['developer_genres'] = data['developer'].map(data.groupby('developer')['genres'].apply(lambda x: list(set([item for sublist in x for item in sublist]))))
data['developer_categories'] = data['developer'].map(data.groupby('developer')['categories'].apply(lambda x: list(set([item for sublist in x for item in sublist]))))
data['publisher_genres'] = data['publisher'].map(data.groupby('publisher')['genres'].apply(lambda x: list(set([item for sublist in x for item in sublist]))))
data['publisher_categories'] = data['publisher'].map(data.groupby('publisher')['categories'].apply(lambda x: list(set([item for sublist in x for item in sublist]))))

In [ ]:
data['log_price'] = np.log1p(data['price'])
data['log_recommendations'] = np.log1p(data['recommendations'])

In [ ]:
data['is_hit'] = data['recommendations'] > 800
#data['is_mega_hit'] = data['recommendations'] > 1000
data['is_discount'] = (data['price'] % 1) > 0.90
data['developer_hit_ratio'] = data.copy().sort_values('release_date').groupby('developer')['is_hit'].transform(lambda x: x.shift(1).expanding().mean())
data['developer_hit_ratio'] = data['developer_hit_ratio'].fillna(0)
data['publisher_hit_ratio'] = data.copy().sort_values('release_date').groupby('publisher')['is_hit'].transform(lambda x: x.shift(1).expanding().mean())
data['publisher_hit_ratio'] = data['publisher_hit_ratio'].fillna(0)
data['developer_game_number'] = data.copy().sort_values('release_date').groupby('developer').cumcount() + 1
data['publisher_game_number'] = data.copy().sort_values('release_date').groupby('publisher').cumcount() + 1
#data['timeliness'] = data['recommendations'] / ((2026 - data['release_date'].astype(str).str.split('-').str[0].astype(int)) * 12 - (data['release_date'].astype(str).str.split('-').str[1].astype(int) - 1)).replace(0, 1)
data['release_month'] = data['release_date'].dt.month
data['month_sin'] = np.sin(2 * np.pi * data['release_month'] / 12)
data['month_cos'] = np.cos(2 * np.pi * data['release_month'] / 12)
data['total_niches_count'] = data.groupby('developer')['niche'].transform(lambda x: x.shift(1).expanding().apply(lambda y: len(np.unique(y.dropna())))).fillna(0)
data['is_specialist'] = (data['total_niches_count'] <= 2).astype(int)

data['developer_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['developer_genres'])]
data['developer_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['developer_categories'])]
data['publisher_genre_overlap'] = [len(set(g) & set(h)) / len(set(g)) if len(g) > 0 else 0 for g, h in zip(data['genres'], data['publisher_genres'])]
data['publisher_category_overlap'] = [len(set(c) & set(h)) / len(set(c)) if len(c) > 0 else 0 for c, h in zip(data['categories'], data['publisher_categories'])]
data['publisher_niche_count'] = data.groupby(['publisher', 'niche'])['name'].transform('count')
data['developer_niche_count'] = data.groupby(['developer', 'niche'])['name'].transform('count')
data['developer_std'] = data.groupby('developer')['price'].transform(lambda x: x.shift(1).expanding().std()).fillna(0)
data['publisher_std'] = data.groupby('publisher')['price'].transform(lambda x: x.shift(1).expanding().std()).fillna(0)
data['price_per_genre'] = data['log_price'] / (data['genre_count'] + 1)

data['developer_price'] = data.groupby('developer')['price'].transform('mean')
data['publisher_price'] = data.groupby('publisher')['price'].transform('mean')
data['developer_recommendations'] = data.groupby('developer')['recommendations'].transform('mean')
data['publisher_recommendations'] = data.groupby('publisher')['recommendations'].transform('mean')
#data['quality'] = data['recommendations'] / (data['price'] + 0.001)
#data['developer_quality_ratio'] = data.groupby('developer')['quality'].transform('mean')
#data['publisher_quality_ratio'] = data.groupby('publisher')['quality'].transform('mean')
data['developer_momentum'] = data.copy().sort_values(['developer', 'release_date']).groupby('developer')['recommendations'].transform(lambda x: x.diff().shift(1)).fillna(0)
#data['developer_niche_recommendations'] = data.groupby(['developer', 'niche'])['recommendations'].transform('mean')
data['publisher_momentum'] = data.copy().sort_values(['publisher', 'release_date']).groupby('publisher')['recommendations'].transform(lambda x: x.diff().shift(1)).fillna(0)
#data['publisher_niche_recommendations'] = data.groupby(['publisher', 'niche'])['recommendations'].transform('mean')

In [ ]:
#data['log_developer_price'] = np.log1p(data['developer_price'])
data['log_publisher_price'] = np.log1p(data['publisher_price'])
data['log_developer_recommendations'] = np.log1p(data['developer_recommendations'])
data['log_publisher_recommendations'] = np.log1p(data['publisher_recommendations'])
#data['log_quality'] = np.log1p(data['quality'])
#data['log_developer_quality_ratio'] = np.log1p(data['developer_quality_ratio'])
#data['log_publisher_quality_ratio'] = np.log1p(data['publisher_quality_ratio'])
data['log_developer_momentum'] = np.sign(data['developer_momentum']) * np.log1p(np.abs(data['developer_momentum']))
#data['log_developer_niche_recommendations'] = np.log1p(data['developer_niche_recommendations'])
data['log_publisher_momentum'] = np.sign(data['publisher_momentum']) * np.log1p(np.abs(data['publisher_momentum']))
#data['log_publisher_niche_recommendations'] = np.log1p(data['publisher_niche_recommendations'])

data['developer_count'] = data['developer'].map(data['developer'].value_counts())
data['log_developer_count'] = np.log1p(data['developer_count'])
data['publisher_count'] = data['publisher'].map(data['publisher'].value_counts())
data['log_publisher_count'] = np.log1p(data['publisher_count'])

In [ ]:
data.head()

In [ ]:
#inf_counts = np.isinf(data.select_dtypes(include=[np.number])).sum()
#print(inf_counts[inf_counts > 0])
#print('----------------')
#nan_counts = data.isnull().sum()
#print(nan_counts[nan_counts > 0])

In [ ]:
robust_cols = [
    'log_price', 'log_publisher_price', 
    'log_developer_recommendations', 'log_publisher_recommendations', 
#    'log_quality', 'log_developer_quality_ratio', 'log_publisher_quality_ratio', 'log_developer_price',
    'log_developer_momentum', 'log_publisher_momentum',
#    'log_developer_niche_recommendations', 'log_publisher_niche_recommendations',
    'developer_niche_count', 'publisher_niche_count',
    'log_developer_count', 'log_publisher_count'
]
minmax_cols = [
#    'timeliness', 
    'genre_count', 'categories_count', 
    'developer_std', 'publisher_std', 
    'unique_genres', 'unique_categories',
    'developer_genre_overlap', 'developer_category_overlap',
    'publisher_genre_overlap', 'publisher_category_overlap'
]
robust = RobustScaler()
data[robust_cols] = robust.fit_transform(data[robust_cols])
mms = MinMaxScaler()
data[minmax_cols] = mms.fit_transform(data[minmax_cols])

In [ ]:
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(data['genres'])
genre_df = pd.DataFrame(genre_encoded, columns=mlb.classes_, index=data.index)
category_encoded = mlb.fit_transform(data['categories'])
category_df = pd.DataFrame(category_encoded, columns=mlb.classes_, index=data.index)
data = pd.concat([data, genre_df, category_df], axis=1)
data['mobile'] = ((data['Remote Play on Phone'] == 1) | (data['Remote Play on Tablet'] == 1)).astype(int)

In [ ]:
data = data.drop([
    'publisher', 'developer', 'publisher_count', 'developer_count', 'Remote Play on Tablet', 
    'price', 'recommendations', 'genres', 'categories', 'name', 'niche', 'Remote Play on Phone',
    'publisher_genres', 'publisher_categories', 'developer_genres',  'appid',
    'developer_momentum', 'publisher_momentum', 'developer_categories', 'release_date',
    'publisher_price', 'developer_price', 'developer_recommendations', 'log_developer_recommendations',
    'publisher_recommendations', 'log_publisher_price', 'log_publisher_recommendations',
    'Family Sharing', 'PvP', 'Co-op', 'Shared/Split Screen', 'release_month'
], axis=1)

In [ ]:
corr_matrix = data.corr().abs()

# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find features with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]

print(f"Redundant columns to drop: {to_drop}")
# data = data.drop(columns=to_drop)

In [ ]:
# Unstack the matrix to get pairs
pairs = upper.unstack().dropna()
# Sort to see the highest correlations at the top
sorted_pairs = pairs.sort_values(ascending=False)

print("Top 10 most redundant pairs:")
print(sorted_pairs.head(10))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Replace 'col1' and 'col2' with names from your 'to_drop' list
sns.scatterplot(x=data['publisher_std'], y=data['developer_std'])
plt.title("Visualizing Redundancy")
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
# Plotting only a subset of columns if you have 139 (it will be too crowded otherwise)
subset = data.iloc[:, :30] 
sns.heatmap(subset.corr(), annot=False, cmap='coolwarm')
plt.show()

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
plot_data = data.explode('genres').explode('categories')
sns.barplot(data=plot_data, x='release_year', y='price')
plt.show()
sns.barplot(data=plot_data, x='release_year', y='recommendations')
plt.show()

In [ ]:
timeline_data = plot_data.groupby(['release_year', 'genres']).size().reset_index(name='count')
timeline_data_2 = plot_data.groupby(['release_year', 'categories']).size().reset_index(name='count')
sns.lineplot(data=timeline_data, x='release_year', y='count', hue='genres')
plt.show()
sns.lineplot(data=timeline_data_2, x='release_year', y='count', hue='categories')
plt.show()

In [ ]:
sns.barplot(data=plot_data, x='price', y='genres')
plt.show()
sns.barplot(data=plot_data, x='price', y='categories')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='genres')
plt.show()
sns.barplot(data=plot_data, x='recommendations', y='categories')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.histplot(data['price'], kde=True)
plt.show()
sns.histplot(data['recommendations'], kde=True)
plt.show()
sns.histplot(data['log_price'], kde=True)
plt.show()
sns.histplot(data['log_recommendations'], kde=True)
plt.show()